# This is the notebook for my capstone project in Coursera Data Science

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

## Week 3: Segmenting and Clustering Neighborhoods in Toronto

### Download data

In [2]:
DF_list=pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Toronto_DF=DF_list[0]

### Drop rows with no assigned borough

In [3]:
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'Not assigned']

### Separate comma deliminated Neighborhoods into separate rows

In [4]:
# convert to list
Toronto_DF['Neighborhood'] = Toronto_DF['Neighborhood'].str.split(', ')

# convert 'Neigborhood' list of pd.Series then stack it
Toronto_DF = (Toronto_DF
 .set_index(['Postal Code','Borough'])['Neighborhood']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'Neighborhood'}))

C:\Users\Marylin Huff\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
Toronto_DF=Toronto_DF.drop('level_2',axis=1)
Toronto_DF.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


In [6]:
Toronto_DF.shape

(217, 3)

### Retrieve Longitude and Latitude

In [7]:
DF_coord=pd.read_csv('http://cocl.us/Geospatial_data')
DF_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
Toronto_DF=pd.merge(Toronto_DF,DF_coord, on ='Postal Code')
Toronto_DF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


### Now that I have the coordinates with the Neighborhoods, I can start exploring

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with the Neighborhoods superimposed

In [11]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_DF['Latitude'], Toronto_DF['Longitude'], Toronto_DF['Borough'], Toronto_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### I'm going to limit the anaysis to only Neighborhoods whose name contains 'P'

In [12]:
TorontoTrimmed_DF = Toronto_DF[Toronto_DF['Neighborhood'].str.contains('P')]
TorontoTrimmed_DF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
13,M4B,East York,Parkview Hill,43.706397,-79.309937


In [13]:
TorontoTrimmed_DF.shape

(38, 5)

In [14]:
# create a trimmed map of Toronto using latitude and longitude values
map_TorontoTrimmed = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoTrimmed_DF['Latitude'], TorontoTrimmed_DF['Longitude'], TorontoTrimmed_DF['Borough'], TorontoTrimmed_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TorontoTrimmed)  
    
map_TorontoTrimmed

## Defining Foursquare creditials

In [15]:
CLIENT_ID = '3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD' # your Foursquare ID
CLIENT_SECRET = 'QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0' # your Foursquare Secret
VERSION = '20200625'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD
CLIENT_SECRET:QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0


## Create a function to explore the venues near each neighborhood

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the code

In [17]:
Toronto_venues = getNearbyVenues(names=TorontoTrimmed_DF['Neighborhood'],
                                   latitudes=TorontoTrimmed_DF['Latitude'],
                                   longitudes=TorontoTrimmed_DF['Longitude']
                                  )

Parkwoods
Regent Park
Queen's Park
Ontario Provincial Government
Parkview Hill
West Deane Park
Princess Gardens
Port Union
Berczy Park
Thorncliffe Park
Northwood Park
Little Portugal
Kennedy Park
East Birchmount Park
Parkdale Village
Exhibition Place
North Park
Maple Leaf Park
Upwood Park
Bedford Park
Lawrence Park
Dorset Park
Forest Hill Road Park
High Park
Lawrence Park
Parkdale
Canada Post Gateway Processing Centre
St. Phillips
Moore Park
Grange Park
Deer Park
Stn A PO Boxes
First Canadian Place
Business reply mail Processing Centre
South Central Letter Processing Plant Toronto
King's Mill Park
Kingsway Park South East
Kingsway Park South West


In [18]:
print(Toronto_venues.shape)
Toronto_venues.head()

(759, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Regent Park,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
3,Regent Park,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
4,Regent Park,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


### How many venues are near each Neighborhood?

In [19]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bedford Park,24,24,24,24,24,24
Berczy Park,58,58,58,58,58,58
Business reply mail Processing Centre,17,17,17,17,17,17
Canada Post Gateway Processing Centre,12,12,12,12,12,12
Deer Park,16,16,16,16,16,16
Dorset Park,6,6,6,6,6,6
East Birchmount Park,5,5,5,5,5,5
Exhibition Place,24,24,24,24,24,24
First Canadian Place,100,100,100,100,100,100


### How many unique categories are there?

In [20]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 175 uniques categories.


# Analyzing each Neighborhood

In [21]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# Create function to reorder columns
def reorder_columns(columns, first_cols=[]):
    columns=list(set(columns)-set(first_cols))
    new_order=first_cols+columns
    return new_order

# move Neighborhood to first column
my_list=Toronto_onehot.columns.tolist()
reordered_cols=reorder_columns(my_list, first_cols=['Neighborhood'])
Toronto_onehot=Toronto_onehot[reordered_cols]

Toronto_onehot.head()

,Neighborhood,Gym,Gym / Fitness Center,Swim School,Athletics & Sports,Building,Gluten-free Restaurant,Filipino Restaurant,Beer Store,Warehouse Store,Yoga Studio,Food & Drink Shop,Supplement Shop,Burger Joint,Church,Diner,Smoothie Shop,Indian Restaurant,Poutine Place,Skate Park,Comic Shop,French Restaurant,Construction & Landscaping,Colombian Restaurant,Cocktail Bar,Intersection,Caribbean Restaurant,Hotel,Bistro,Café,Italian Restaurant,Mobile Phone Shop,Museum,Juice Bar,Spa,Plaza,Chocolate Shop,Belgian Restaurant,Beer Bar,Tanning Salon,Bus Line,Salad Place,Sporting Goods Shop,Stadium,Soup Place,Liquor Store,Movie Theater,Jazz Club,Mediterranean Restaurant,Trail,Miscellaneous Shop,Gift Shop,Recording Studio,Art Gallery,Basketball Stadium,Malay Restaurant,Nightclub,College Cafeteria,Doner Restaurant,Event Space,Burrito Place,Dog Run,Irish Pub,Cupcake Shop,Convenience Store,Distribution Center,Fried Chicken Joint,Molecular Gastronomy Restaurant,Hardware Store,BBQ Joint,Jewelry Store,Office,Garden,Supermarket,Breakfast Spot,Wine Shop,Sports Bar,New American Restaurant,Organic Grocery,Bed & Breakfast,Furniture / Home Store,Basketball Court,Cajun / Creole Restaurant,Greek Restaurant,Train Station,Japanese Restaurant,Mexican Restaurant,Garden Center,Food Court,Light Rail Station,Tea Room,Chinese Restaurant,College Auditorium,Hospital,Gaming Cafe,Beach,Mac & Cheese Joint,Brazilian Restaurant,Performing Arts Venue,Seafood Restaurant,Cheese Shop,Antique Shop,Bar,Shopping Mall,Baseball Field,Fish Market,Deli / Bodega,Thrift / Vintage Store,Sushi Restaurant,Grocery Store,Hobby Shop,Massage Studio,Clothing Store,Electronics Store,Fast Food Restaurant,Brewery,Shoe Store,Falafel Restaurant,Flea Market,Bakery,Monument / Landmark,Wine Bar,Department Store,Food Truck,Dessert Shop,Rental Car Location,Music Venue,Comfort Food Restaurant,Poke Place,Salon / Barbershop,Bagel Shop,Sandwich Place,Ice Cream Shop,Historic Site,Pub,Lounge,Optical Shop,Eastern European Restaurant,Opera House,Steakhouse,Wings Joint,Park,Vietnamese Restaurant,Concert Hall,Cosmetics Shop,Pet Store,American Restaurant,Bookstore,Cuban Restaurant,Discount Store,Korean Restaurant,Theater,Arts & Crafts Store,Restaurant,Bank,Speakeasy,Asian Restaurant,Gastropub,Pizza Place,Creperie,Record Shop,Butcher,Middle Eastern Restaurant,Auto Workshop,Fountain,Climbing Gym,Gourmet Shop,Pharmacy,Men's Store,Gas Station,Vegetarian / Vegan Restaurant,Tailor Shop,Thai Restaurant,Farmers Market,Coffee Shop,General Travel
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
Toronto_onehot.shape

(759, 176)

### Grouping by Neighborhood

In [23]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Gym,Gym / Fitness Center,Swim School,Athletics & Sports,Building,Gluten-free Restaurant,Filipino Restaurant,Beer Store,Warehouse Store,Yoga Studio,Food & Drink Shop,Supplement Shop,Burger Joint,Church,Diner,Smoothie Shop,Indian Restaurant,Poutine Place,Skate Park,Comic Shop,French Restaurant,Construction & Landscaping,Colombian Restaurant,Cocktail Bar,Intersection,Caribbean Restaurant,Hotel,Bistro,Café,Italian Restaurant,Mobile Phone Shop,Museum,Juice Bar,Spa,Plaza,Chocolate Shop,Belgian Restaurant,Beer Bar,Tanning Salon,Bus Line,Salad Place,Sporting Goods Shop,Stadium,Soup Place,Liquor Store,Movie Theater,Jazz Club,Mediterranean Restaurant,Trail,Miscellaneous Shop,Gift Shop,Recording Studio,Art Gallery,Basketball Stadium,Malay Restaurant,Nightclub,College Cafeteria,Doner Restaurant,Event Space,Burrito Place,Dog Run,Irish Pub,Cupcake Shop,Convenience Store,Distribution Center,Fried Chicken Joint,Molecular Gastronomy Restaurant,Hardware Store,BBQ Joint,Jewelry Store,Office,Garden,Supermarket,Breakfast Spot,Wine Shop,Sports Bar,New American Restaurant,Organic Grocery,Bed & Breakfast,Furniture / Home Store,Basketball Court,Cajun / Creole Restaurant,Greek Restaurant,Train Station,Japanese Restaurant,Mexican Restaurant,Garden Center,Food Court,Light Rail Station,Tea Room,Chinese Restaurant,College Auditorium,Hospital,Gaming Cafe,Beach,Mac & Cheese Joint,Brazilian Restaurant,Performing Arts Venue,Seafood Restaurant,Cheese Shop,Antique Shop,Bar,Shopping Mall,Baseball Field,Fish Market,Deli / Bodega,Thrift / Vintage Store,Sushi Restaurant,Grocery Store,Hobby Shop,Massage Studio,Clothing Store,Electronics Store,Fast Food Restaurant,Brewery,Shoe Store,Falafel Restaurant,Flea Market,Bakery,Monument / Landmark,Wine Bar,Department Store,Food Truck,Dessert Shop,Rental Car Location,Music Venue,Comfort Food Restaurant,Poke Place,Salon / Barbershop,Bagel Shop,Sandwich Place,Ice Cream Shop,Historic Site,Pub,Lounge,Optical Shop,Eastern European Restaurant,Opera House,Steakhouse,Wings Joint,Park,Vietnamese Restaurant,Concert Hall,Cosmetics Shop,Pet Store,American Restaurant,Bookstore,Cuban Restaurant,Discount Store,Korean Restaurant,Theater,Arts & Crafts Store,Restaurant,Bank,Speakeasy,Asian Restaurant,Gastropub,Pizza Place,Creperie,Record Shop,Butcher,Middle Eastern Restaurant,Auto Workshop,Fountain,Climbing Gym,Gourmet Shop,Pharmacy,Men's Store,Gas Station,Vegetarian / Vegan Restaurant,Tailor Shop,Thai Restaurant,Farmers Market,Coffee Shop,General Travel
0,Bedford Park,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.083333,0.000000,0.000000,0.041667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.041667,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.0000

In [24]:
Toronto_grouped.shape

(35, 176)

### What are the five most common venues in each neighborhood?

In [25]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bedford Park----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3          Restaurant  0.08
4       Grocery Store  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3              Bakery  0.03
4          Restaurant  0.03


----Business reply mail Processing Centre----
                  venue  freq
0  Fast Food Restaurant  0.06
1           Yoga Studio  0.06
2            Skate Park  0.06
3                  Park  0.06
4               Butcher  0.06


----Canada Post Gateway Processing Centre----
                       venue  freq
0                Coffee Shop  0.17
1                      Hotel  0.17
2                        Gym  0.08
3  Middle Eastern Restaurant  0.08
4             Sandwich Place  0.08


----Deer Park----
                 venue  freq
0          Coffee Shop  0.12
1                  Pub  0.12
2     Sushi Restaurant  0.06

### And now a dataframe in descending order for most popular venues

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bedford Park,Restaurant,Italian Restaurant,Sandwich Place,Coffee Shop,Pub,Juice Bar,Pizza Place,Hobby Shop,Butcher,Grocery Store
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Café,Cheese Shop,Restaurant,Bakery,Seafood Restaurant,Bistro,Museum
2,Business reply mail Processing Centre,Comic Shop,Garden,Skate Park,Butcher,Recording Studio,Auto Workshop,Yoga Studio,Restaurant,Light Rail Station,Pizza Place
3,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Gym,Burrito Place,Sandwich Place,Fried Chicken Joint,Gas Station,Mediterranean Restaurant,American Restaurant,Middle Eastern Restaurant
4,Deer Park,Coffee Shop,Pub,Sushi Restaurant,American Restaurant,Liquor Store,Pizza Place,Bagel Shop,Light Rail Station,Bank,Sports Bar


## Clustering the Neighborhoods

In [27]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 3])

### Adding Cluster info

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoTrimmed_DF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Food & Drink Shop,Park,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria,Nightclub
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,French Restaurant,Cosmetics Shop,Antique Shop
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
13,M4B,East York,Parkview Hill,43.706397,-79.309937,0.0,Pizza Place,Intersection,Gym / Fitness Center,Pet Store,Athletics & Sports,Pharmacy,Café,Bank,Breakfast Spot,Gastropub


In [29]:
Toronto_merged.dropna(inplace=True)
Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Food & Drink Shop,Park,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria,Nightclub
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,French Restaurant,Cosmetics Shop,Antique Shop
6,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
7,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
13,M4B,East York,Parkview Hill,43.706397,-79.309937,0.0,Pizza Place,Intersection,Gym / Fitness Center,Pet Store,Athletics & Sports,Pharmacy,Café,Bank,Breakfast Spot,Gastropub


# Visualizing it!!!!

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the Clusters

In [37]:
# Cluster 1 (cluster label = 0)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Regent Park,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,French Restaurant,Cosmetics Shop,Antique Shop
6,Queen's Park,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
7,Ontario Provincial Government,0.0,Coffee Shop,Diner,Sushi Restaurant,Gym,Park,Theater,Fried Chicken Joint,Distribution Center,Bar,Burrito Place
13,Parkview Hill,0.0,Pizza Place,Intersection,Gym / Fitness Center,Pet Store,Athletics & Sports,Pharmacy,Café,Bank,Breakfast Spot,Gastropub
38,Berczy Park,0.0,Coffee Shop,Cocktail Bar,Beer Bar,Café,Cheese Shop,Restaurant,Bakery,Seafood Restaurant,Bistro,Museum
49,Thorncliffe Park,0.0,Indian Restaurant,Gym,Middle Eastern Restaurant,Grocery Store,Discount Store,Restaurant,Bank,Sandwich Place,Fast Food Restaurant,Park
59,Northwood Park,0.0,Massage Studio,Coffee Shop,Caribbean Restaurant,Furniture / Home Store,Bar,Falafel Restaurant,Nightclub,Irish Pub,Dog Run,Burrito Place
66,Little Portugal,0.0,Bar,Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Men's Store,Vietnamese Restaurant,Café,New American Restaurant,Dog Run,French Restaurant
78,Parkdale Village,0.0,Café,Bakery,Coffee Shop,Breakfast Spot,Gym,Performing Arts Venue,Stadium,Nightclub,Italian Restaurant,Burrito Place
79,Exhibition Place,0.0,Café,Bakery,Coffee Shop,Breakfast Spot,Gym,Performing Arts Venue,Stadium,Nightclub,Italian Restaurant,Burrito Place


In [39]:
# Cluster 2 (cluster label = 1)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
205,King's Mill Park,1.0,Baseball Field,General Travel,Distribution Center,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria
211,Kingsway Park South East,1.0,Baseball Field,General Travel,Distribution Center,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria


In [38]:
# Cluster 3 (cluster label = 2)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Kennedy Park,2.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,General Travel,Basketball Stadium,Dog Run,Burrito Place,Event Space,Doner Restaurant
70,East Birchmount Park,2.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,General Travel,Basketball Stadium,Dog Run,Burrito Place,Event Space,Doner Restaurant


In [40]:
# Cluster 4 (cluster label = 3)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,3.0,Food & Drink Shop,Park,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria,Nightclub
124,Forest Hill Road Park,3.0,Sushi Restaurant,Trail,Jewelry Store,Park,General Travel,Nightclub,Irish Pub,Dog Run,Burrito Place,Event Space
153,Moore Park,3.0,Gym,Trail,Malay Restaurant,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria


In [41]:
# Cluster 5 (cluster label = 4)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Port Union,4.0,Construction & Landscaping,Bar,Fried Chicken Joint,Convenience Store,Cupcake Shop,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant
90,North Park,4.0,Construction & Landscaping,Park,Basketball Court,Bakery,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria
91,Maple Leaf Park,4.0,Construction & Landscaping,Park,Basketball Court,Bakery,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria
92,Upwood Park,4.0,Construction & Landscaping,Park,Basketball Court,Bakery,Irish Pub,Dog Run,Burrito Place,Event Space,Doner Restaurant,College Cafeteria
